<a href="https://colab.research.google.com/github/yoosoo13/yoosoo13/blob/main/%ED%98%BC%EA%B3%B5%ED%8C%8C/%ED%98%BC%EA%B3%B5%ED%8C%8C2_%EC%84%A0%ED%83%9D%EB%AF%B8%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [선택미션]
## 전체 도서의 쪽수 구하기
- PANDAS DataFrame은 한 행씩 순차적으로 처리하는데 최적화되어 있지 않음
  - DataFrame을 for문으로 반복하는 것은 아주 비효율적
  - DataFrame은 각 행 또는 각 열에 원하는 함수를 자동으로 적용해주는 여러 가지 방법을 제공

In [ ]:
# 혼공파2-API호출에서 저장했던 '20대가 가장 좋아하는 도서 목록'사용하기 위해 불러오기
# https://drive.google.com/uc?id=(파일경로)
import gdown
gdown.download('https://drive.google.com/uc?id=1-4OcgJ9F3ZVfBJGWi9Xe9GmocFDylp0M', '23s_best_book.json', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-4OcgJ9F3ZVfBJGWi9Xe9GmocFDylp0M
To: /content/23s_best_book.json
100%|██████████| 79.5k/79.5k [00:00<00:00, 71.9MB/s]


'23s_best_book.json'

In [ ]:
import pandas as pd
books = pd.read_json('23s_best_book.json')
books.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,bookDtlUrl
0,1,1,"물고기는 존재하지 않는다 :상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기",룰루 밀러 지음 ;정지인 옮김,곰출판,2021,9791189327156,03400,,490.99,자연과학 > 동물학 > 동물학,494,https://image.aladin.co.kr/product/28465/73/co...,https://data4library.kr/bookV?seq=6360937
1,2,2,불편한 편의점 :김호연 장편소설,지은이: 김호연,나무옆의자,2021,9791161571188,03810,,813.7,문학 > 한국문학 > 소설,492,https://image.aladin.co.kr/product/26942/84/co...,https://data4library.kr/bookV?seq=5962217
2,3,3,지구 끝의 온실 :김초엽 장편소설,지은이: 김초엽,Giant Books(자이언트북스),2021,9791191824001,03810,,813.7,문학 > 한국문학 > 소설,466,https://image.aladin.co.kr/product/27692/63/co...,https://data4library.kr/bookV?seq=6065686
3,4,4,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,438,https://image.aladin.co.kr/product/24512/70/co...,https://data4library.kr/bookV?seq=5707051
4,5,5,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,425,https://image.aladin.co.kr/product/19359/16/co...,https://data4library.kr/bookV?seq=5430429


In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
def get_page_cnt(isbn):
  # 1. 온라인 서점(yes24)의 검색 결과 페이지 URL 생성
  url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'

  # 2. requests.get()함수로 검색 결과 페이지의 HTML을 가져오기
  r = requests.get(url.format(isbn))

  # 3. 뷰티풀수프로 HTML을 파싱
  soup = BeautifulSoup(r.text, 'html.parser')

  # 4. 뷰티풀플수프의 find()메서드로 <a>태그를 찾아 상세 페이지 URL 추출
  prd_info = soup.find('a', attrs={'class' : 'gd_name'})

  # 5. requests.get()함수로 다시 도서 상세 페이지의 HTML 가져오기 & 6. 뷰티플수프로 HTML 파싱
  url = 'http://www.yes24.com'+prd_info['href']
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')

  # 7. 뷰티풀수프의 find()메서드로 '품목정보' <div> 태그 찾기
  prd_detail = soup.find('div', attrs={'id' : 'infoset_specific'})

  # 8. 뷰티수프의 findall()메서드로 '쪽수'가 들어있는 <tr> 태그 찾기
  prd_tr_list = prd_detail.find_all('tr')

  # 9. 쪽수가 들어 있는 <th>를 찾아 <td>에 담긴 값을 반환
  for tr in prd_tr_list :
    if tr.find('th').get_text() == '쪽수, 무게, 크기':
      return tr.find('td').get_text().split()[0]
  return ''

In [ ]:
# 첫 번째 도서의 ISBN넣어서 위 함수 확인하기
get_page_cnt(9791189327156)

'300쪽'

#### 데이터프레임 행 혹은 열에 함수 적용하기 : apply()메서드

In [ ]:
# book_df 데이터프레임에는 모두 200개의 도서가 들어 있음. 이 중 가장 인기 있는 10권 확인 : head(10)사용
top10_books = books.head(10)
top10_books

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,bookDtlUrl
0,1,1,"물고기는 존재하지 않는다 :상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기",룰루 밀러 지음 ;정지인 옮김,곰출판,2021,9791189327156,03400,,490.99,자연과학 > 동물학 > 동물학,494,https://image.aladin.co.kr/product/28465/73/co...,https://data4library.kr/bookV?seq=6360937
1,2,2,불편한 편의점 :김호연 장편소설,지은이: 김호연,나무옆의자,2021,9791161571188,03810,,813.7,문학 > 한국문학 > 소설,492,https://image.aladin.co.kr/product/26942/84/co...,https://data4library.kr/bookV?seq=5962217
2,3,3,지구 끝의 온실 :김초엽 장편소설,지은이: 김초엽,Giant Books(자이언트북스),2021,9791191824001,03810,,813.7,문학 > 한국문학 > 소설,466,https://image.aladin.co.kr/product/27692/63/co...,https://data4library.kr/bookV?seq=6065686
3,4,4,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,438,https://image.aladin.co.kr/product/24512/70/co...,https://data4library.kr/bookV?seq=5707051
4,5,5,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,425,https://image.aladin.co.kr/product/19359/16/co...,https://data4library.kr/bookV?seq=5430429
5,6,6,천 개의 파랑 :천선란 장편소설,지은이: 천선란,허블,2020,9791190090261,03810,,813.7,문학 > 한국문학 > 소설,412,https://image.aladin.co.kr/product/24895/69/co...,https://data4library.kr/bookV?seq=5738747
6,7,7,홍학의 자리 :정해연 장편소설,지은이: 정해연,문학동네,2021,9788954681155,03810,,813.7,문학 > 한국문학 > 소설,383,https://image.aladin.co.kr/product/27587/94/co...,https://data4library.kr/bookV?seq=6050062
7,8,8,구의 증명 :최진영 소설,지은이: 최진영,은행나무,2015,9788956608556,03810,,813.7,문학 > 한국문학 > 소설,363,http://image.aladin.co.kr/product/5527/50/cove...,https://data4library.kr/bookV?seq=2619618
8,9,9,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,03810,,813.7,문학 > 한국문학 > 소설,361,https://image.aladin.co.kr/product/24131/37/co...,https://data4library.kr/bookV?seq=5674475
9,10,10,"칵테일, 러브, 좀비",지은이: 조예은,안전가옥,2020,9791190174756,03810,,813.7,문학 > 한국문학 > 소설,356,https://image.aladin.co.kr/product/23736/85/co...,https://data4library.kr/bookV?seq=5650149


In [ ]:
# 인기 10위 도서의 쪽수를 확인하기 전에, 데이터프레임의 한 행을 받아 get_page_cn()함수를 사용해 쪽수를 구하는 함수 생성 (단순히 'isbn13'열의 값을 get_page_cnt()함수로 전달하는 역할만 수행)
def get_page_cnt2(row):
  isbn = row['isbn13']
  return get_page_cnt(isbn)

In [ ]:
# apply()메서드 사용
page_count = top10_books.apply(get_page_cnt2, axis=1)  # apply()메서드에 axis매개변수에 1을 지정하면 행에, 기본값 0을 사용하면 열에 적용함
print(page_count)

0    300쪽
1    268쪽
2    392쪽
3    300쪽
4    330쪽
5    376쪽
6    336쪽
7    177쪽
8    340쪽
9    165쪽
dtype: object


### 데이터프레임과 시리즈 합치기 : merge()함수
- 판다스에서 두 데이터프레임을 합침 & 데이터프레임과 시리즈를 합칠 때, merge()함수 사용
- 두 객체의 인덱스를 기준으로 합칠 경우, left_index와 right_index매개변수를 True로 지정

In [ ]:
page_count.name = 'page_count'
print(page_count)

0    300쪽
1    268쪽
2    392쪽
3    300쪽
4    330쪽
5    376쪽
6    336쪽
7    177쪽
8    340쪽
9    165쪽
Name: page_count, dtype: object


In [ ]:
top10_with_page_count = pd.merge(top10_books, page_count, left_index=True, right_index=True)
top10_with_page_count

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,bookDtlUrl,page_count
0,1,1,"물고기는 존재하지 않는다 :상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기",룰루 밀러 지음 ;정지인 옮김,곰출판,2021,9791189327156,03400,,490.99,자연과학 > 동물학 > 동물학,494,https://image.aladin.co.kr/product/28465/73/co...,https://data4library.kr/bookV?seq=6360937,300쪽
1,2,2,불편한 편의점 :김호연 장편소설,지은이: 김호연,나무옆의자,2021,9791161571188,03810,,813.7,문학 > 한국문학 > 소설,492,https://image.aladin.co.kr/product/26942/84/co...,https://data4library.kr/bookV?seq=5962217,268쪽
2,3,3,지구 끝의 온실 :김초엽 장편소설,지은이: 김초엽,Giant Books(자이언트북스),2021,9791191824001,03810,,813.7,문학 > 한국문학 > 소설,466,https://image.aladin.co.kr/product/27692/63/co...,https://data4library.kr/bookV?seq=6065686,392쪽
3,4,4,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,438,https://image.aladin.co.kr/product/24512/70/co...,https://data4library.kr/bookV?seq=5707051,300쪽
4,5,5,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,425,https://image.aladin.co.kr/product/19359/16/co...,https://data4library.kr/bookV?seq=5430429,330쪽
5,6,6,천 개의 파랑 :천선란 장편소설,지은이: 천선란,허블,2020,9791190090261,03810,,813.7,문학 > 한국문학 > 소설,412,https://image.aladin.co.kr/product/24895/69/co...,https://data4library.kr/bookV?seq=5738747,376쪽
6,7,7,홍학의 자리 :정해연 장편소설,지은이: 정해연,문학동네,2021,9788954681155,03810,,813.7,문학 > 한국문학 > 소설,383,https://image.aladin.co.kr/product/27587/94/co...,https://data4library.kr/bookV?seq=6050062,336쪽
7,8,8,구의 증명 :최진영 소설,지은이: 최진영,은행나무,2015,9788956608556,03810,,813.7,문학 > 한국문학 > 소설,363,http://image.aladin.co.kr/product/5527/50/cove...,https://data4library.kr/bookV?seq=2619618,177쪽
8,9,9,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,03810,,813.7,문학 > 한국문학 > 소설,361,https://image.aladin.co.kr/product/24131/37/co...,https://data4library.kr/bookV?seq=5674475,340쪽
9,10,10,"칵테일, 러브, 좀비",지은이: 조예은,안전가옥,2020,9791190174756,03810,,813.7,문학 > 한국문학 > 소설,356,https://image.aladin.co.kr/product/23736/85/co...,https://data4library.kr/bookV?seq=5650149,165쪽
